In [30]:
import random

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, log_loss, roc_auc_score

from src.dataset import get_dataset
from src.models.dualemb import DualEmbPredictor
from src.models.elog import ELOgPredictor
from src.models.freq import FrequencyMatchPredictor
from src.models.uni import UniformMatchPredictor

In [31]:
ITERATIONS = 1
random.seed(548)
np.random.seed(548)

In [32]:
def determine_target(row):
    if row["team_score"] > row["opponent_score"]:
        return 0
    elif row["team_score"] == row["opponent_score"]:
        return 1
    else:
        return 2

In [33]:
dataset = get_dataset()

/mnt/c/Personal/football-matches-prediction/src/dataset.py:47: DtypeWarning: Columns (11,14,50,102,123,125,129,131,133,135,137,139,141,143,145,147,149,157,158,160,161,163,165,167,169,171,173,184,186,187,205,206) have mixed types. Specify dtype option on import or set low_memory=False.
  mls_df = pd.read_csv("data/mls_matches.csv")


In [34]:
dataset

,team_id,opponent_id,team_at_home,opponent_at_home,team_score,opponent_score,fold
0,Scotland,England,1.0,0.0,0,0,international
1,England,Scotland,1.0,0.0,4,2,international
2,Scotland,England,1.0,0.0,2,1,international
3,England,Scotland,1.0,0.0,2,2,international
4,Scotland,England,1.0,0.0,3,0,international
...,...,...,...,...,...,...,...
143000,Sport Lisboa e Benfica,Sporting Clube de Braga,1.0,0.0,3,0,europe
143001,Panathinaikos Athlitikos Omilos,APS Atromitos Athinon,1.0,0.0,2,1,europe
143002,Fulham Football Club,Watford FC,1.0,0.0,4,1,europe
143003,Panthessalonikios Athlitikos Omilos Konstantin...,Athlitiki Enosi Konstantinoupoleos,1.0,0.0,1,1,europe


In [35]:
model_classes = [
    # FrequencyMatchPredictor,
    # UniformMatchPredictor,
    # ELOgPredictor,
    DualEmbPredictor,
]
# folds_names = ["brazil", "libertadores", "mls", "europe", "international"]
folds_names = ["brazil", "international"]

In [36]:
folds_train = [dataset[dataset["fold"] != name] for name in folds_names]
folds_test = [dataset[dataset["fold"] == name] for name in folds_names]

In [37]:
results = pd.DataFrame({}, columns=["metric", "model", "fold", "iteration", "value"])

In [38]:
for iteration in range(ITERATIONS):
    for model_class in model_classes:
        for fold_train, fold_test, fold_test_name in zip(
            folds_train, folds_test, folds_names
        ):
            X_train = fold_train[
                ["team_id", "opponent_id", "team_at_home", "opponent_at_home"]
            ]
            y_train = fold_train[["team_score", "opponent_score"]] / 10.0
            X_test = fold_test[
                ["team_id", "opponent_id", "team_at_home", "opponent_at_home"]
            ]
            y_test = fold_test[["team_score", "opponent_score"]] / 10.0
            model = model_class(
                embedding_dim=10, hidden_dim=2, num_epochs=25, update_learning_rate=0.01
            )
            model.fit(X_train, y_train)
            pred = model.predict_and_update(X_test, y_test)
            max_pred = np.argmax(pred, axis=1)
            target = fold_test.apply(determine_target, axis=1).to_numpy()
            report = classification_report(
                target, max_pred, target_names=["win", "draw", "loss"], output_dict=True
            )
            metrics = {
                "accuracy": report["accuracy"],
                "log_loss": log_loss(target, pred, labels=[0, 1, 2]),
                "micro_auc_roc": roc_auc_score(
                    target, pred, average="micro", multi_class="ovr"
                ),
                "weighted_precision": report["weighted avg"]["precision"],
                "weighted_recall": report["weighted avg"]["recall"],
                "macro_precision": report["macro avg"]["precision"],
                "macro_recall": report["macro avg"]["recall"],
            }
            for key, value in metrics.items():
                results.loc[len(results)] = {
                    "metric": key,
                    "model": model_class.__name__,
                    "fold": fold_test_name,
                    "iteration": iteration + 1,
                    "value": value,
                }

Epoch 1/25


100%|█████████████████████████████████████████████████████████████████| 3699/3699 [00:14<00:00, 254.62it/s, loss=0.0220]


Epoch 2/25


100%|█████████████████████████████████████████████████████████████████| 3699/3699 [00:17<00:00, 216.55it/s, loss=0.0220]


Epoch 3/25


...

100%|█████████████████████████████████████████████████████████████████| 2976/2976 [00:17<00:00, 174.39it/s, loss=0.0110]


Epoch 24/25


100%|█████████████████████████████████████████████████████████████████| 2976/2976 [00:14<00:00, 199.39it/s, loss=0.0110]


Epoch 25/25


100%|█████████████████████████████████████████████████████████████████| 2976/2976 [00:12<00:00, 239.88it/s, loss=0.0109]
/mnt/c/Personal/football-matches-prediction/src/models/dualemb.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pair_X["team_id"] = [0, 1]
/mnt/c/Personal/football-matches-prediction/src/models/dualemb.py:306: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pair_X["opponent_id"] = [1, 0]
/home/razevedo/.cache/pypoetry/virtualenvs/football-matches-prediction-ljJPvyK7-py3.12/lib/python3.

In [39]:
results

,metric,model,fold,iteration,value
0,accuracy,DualEmbPredictor,brazil,1,0.481029
1,log_loss,DualEmbPredictor,brazil,1,1.096001
2,micro_auc_roc,DualEmbPredictor,brazil,1,0.542101
3,weighted_precision,DualEmbPredictor,brazil,1,0.231389
4,weighted_recall,DualEmbPredictor,brazil,1,0.481029
5,macro_precision,DualEmbPredictor,brazil,1,0.160343
6,macro_recall,DualEmbPredictor,brazil,1,0.333333
7,accuracy,DualEmbPredictor,international,1,0.490215
8,log_loss,DualEmbPredictor,international,1,1.056131
9,micro_auc_roc,DualEmbPredictor,international,1,0.637642


In [40]:
results.groupby(["metric", "model", "fold"])["value"].mean().reset_index().groupby(
    ["metric", "model"]
)["value"].mean().reset_index().pivot(index="model", columns="metric", values="value")

metric,accuracy,log_loss,macro_precision,macro_recall,micro_auc_roc,weighted_precision,weighted_recall
model,,,,,,,
DualEmbPredictor,0.485622,1.076066,0.161874,0.333333,0.589872,0.23585,0.485622
